In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import glob
import re
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Get all NetCDF files
nc_files = sorted(glob.glob("2021/RDEFT4_*.nc"))  # Adjust path if necessary

# Extract dates from filenames
def extract_date(filename):
    match = re.search(r"RDEFT4_(\d{8})", filename)
    return pd.to_datetime(match.group(1), format="%Y%m%d") if match else None

# Store data
dates = []
mean_thickness = []

for file_path in nc_files:
    date = extract_date(file_path)
    
    if date:
        ds = xr.open_dataset(file_path)
        sea_ice = ds["sea_ice_thickness"]
        sea_ice_valid = sea_ice.where((sea_ice >= 0) & (sea_ice != -9999) & (sea_ice != -999))
        
        # Compute mean thickness
        mean_value = sea_ice_valid.mean().item()
        mean_thickness.append(mean_value)
        dates.append(date)

# Convert to Pandas DataFrame
df = pd.DataFrame({"Date": dates, "Mean_Thickness": mean_thickness}).sort_values("Date")



In [3]:
# Define output filename
output_filename = "2021/sea_ice_thickness_trend_2021.png"

# Create the plot
plt.figure(figsize=(10, 5))
plt.plot(df["Date"], df["Mean_Thickness"], marker="o", linestyle="-", color="b", label="Observed")

plt.xlabel("Date")
plt.ylabel("Mean Sea Ice Thickness (Meters)")
plt.title("Trend of Sea Ice Thickness in 2021")
plt.legend()
plt.grid()
plt.xticks(rotation=45)

# Fix the y-axis from 0 to 4 meters
plt.ylim(1.3, 2.3)
plt.xlim(pd.to_datetime("2020-10-14"), pd.to_datetime("2021-04-30"))
# Save the plot
plt.savefig(output_filename, bbox_inches="tight", dpi=300)
plt.close()  # Free memory

print(f"Saved: {output_filename}")


Saved: 2021/sea_ice_thickness_trend_2021.png
